In [1]:
import Ipynb_importer
import cv2 as cv
import sys,os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from numba import jit

from image_preprocessing import cdi_process,calc_entropy
from sklearn.preprocessing import StandardScaler,MinMaxScaler

%matplotlib inline
sns.set_style('darkgrid')

Home_os='C:/Users/jingy/Jupyter/Crime/Fire/Dataset/video/'
negative_os=Home_os+'negative/'
positive_os=Home_os+'positive/'


importing Jupyter notebook from C:\Users\jingy\Jupyter\Crime\Combine\CDI_Based\Intensity\Preprocessing\image_preprocessing\cdi_process.ipynb
importing Jupyter notebook from C:\Users\jingy\Jupyter\Crime\Combine\CDI_Based\Intensity\Preprocessing\image_preprocessing\calc_entropy.ipynb


In [2]:
video=os.listdir(negative_os)

for i in video:
    video_name=negative_os+i
    #print(video_name)
    cap = cv.VideoCapture(video_name)
    num=cap.get(7)
    if num!=30:
        print(i,num)
    while 1:
        ret,frame=cap.read() 
        if ret:
            cv.imshow('frame',frame)
            k = cv.waitKey(1) 
            if k ==27:     
                break 
        else:
            break
cv.destroyAllWindows() 

In [3]:
video=os.listdir(negative_os)
#My_CDI=cdi_process.CDI_lib(cap,Threshold=True,mythold=[0,255])
Intensity_list=[]
for i in video:
    start=0
    video_name=negative_os+i
    cap = cv.VideoCapture(video_name)
    My_CDI=cdi_process.CDI_lib(cap,Threshold=True,mythold=[0,255])
    in_list=[]
    while 1:
        if start<=27:
            cdi,frame=My_CDI.CDI(start)
            Intensity=My_CDI.CDI_Intensity(cdi,Normalization=False)
            in_list.append(Intensity)
            cv.imshow('frame',frame)
            cv.imshow('cdi',cdi)
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    Intensity_list.append(in_list)
    cap.release()
cv.destroyAllWindows() 

In [4]:
print(len(Intensity_list))

intensity_dict={'intensity':Intensity_list}
df=pd.DataFrame(intensity_dict)
#df['label']=0
display(df)
print(type(df['intensity']))
# k=df['intensity'].values
# g=[len(k[i]) for i in range(800)]
# print(g)

126


,intensity
0,"[3437145, 3247680, 2615790]"
1,[3431790]
2,[7563300]
3,[2786385]
4,[5978220]
...,...
121,"[5112750, 4932975, 5271105]"
122,"[805800, 1139085, 1172235]"
123,"[3363195, 4137885, 2948055, 3473610]"
124,"[8940555, 5493210, 5044665]"


<class 'pandas.core.series.Series'>


In [5]:
video=os.listdir(positive_os)
#My_CDI=cdi_process.CDI_lib(cap,Threshold=True,mythold=[0,255])
P_Intensity_list=[]
for i in video:
    start=0
    video_name=positive_os+i
    cap = cv.VideoCapture(video_name)
    My_CDI=cdi_process.CDI_lib(cap,Threshold=True,mythold=[0,255])
    in_list=[]
    while 1:
        if start<=27:
            cdi,frame=My_CDI.CDI(start)
            Intensity=My_CDI.CDI_Intensity(cdi,Normalization=False)
            in_list.append(Intensity)
            cv.imshow('frame',frame)
            cv.imshow('cdi',cdi)
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    P_Intensity_list.append(in_list)
    cap.release()
cv.destroyAllWindows() 

In [6]:
print(len(P_Intensity_list))
P_intensity_dict={'intensity':P_Intensity_list}
df=pd.DataFrame(P_intensity_dict)
#df['label']=0
display(df)
print(type(df['intensity']))
# k=df['intensity'].values
# g=[len(k[i]) for i in range(800)]
# print(g)

142


,intensity
0,"[2924850, 2479365, 2073150, 1790100]"
1,[4407675]
2,[3993300]
3,[7957275]
4,[6705225]
...,...
137,"[6768975, 6734295, 6444105]"
138,"[786165, 1313760]"
139,[105315]
140,"[10620750, 10711530]"


<class 'pandas.core.series.Series'>


In [7]:

Combine_Intensity=Intensity_list+P_Intensity_list
min_max_scaler =MinMaxScaler()
for i in range(len(Combine_Intensity)):
    Combine_Intensity[i] = ((min_max_scaler.fit_transform(np.array(Combine_Intensity[i]).reshape(-1,1)))).reshape(-1).tolist()
new_list=[]
for i in Combine_Intensity:
        str1 = ' '.join([str(x) for x in i])
        new_list.append(str1)

print(len(new_list))


268


In [8]:
x = [0] * 126
y =[1]*142
z=x+y

In [9]:
combine_dict={'intensity':new_list,'label':z}
df=pd.DataFrame(combine_dict)

display(df)
y=df['label'].values

,intensity,label
0,1.0 0.7693262961813101 0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0
...,...,...
263,1.0 0.8932496075353242 0.0,1
264,0.0 0.9999999999999998,1
265,0.0,1
266,0.0 1.0,1


In [10]:
data_df = df['intensity'].str.split(' ', expand=True)

data_df['Class'] =y
data_df.to_csv('C:/Users/jingy/Jupyter/Crime/Fire/CDI_Based/Intensity/CSV/Gray.csv')
display(data_df)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,Class
0,1.0,0.7693262961813101,0.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
1,0.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
2,0.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
3,0.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
4,0.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,1.0,0.8932496075353242,0.0,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
264,0.0,0.9999999999999998,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
265,0.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
266,0.0,1.0,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
